In [1]:
# code block for imports 
import pandas as pd 
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import os
import matplotlib.pyplot as plt
import pandas as pd
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
conn = sqlite3.connect('cs506MAPC.db')

## Display first 10 Providers 

In [ ]:
qrystr ="SELECT DISTINCT t1.ProviderName,t1.MaxAdDown,t1.MaxAdUp, t2.town, t3.Longitude, t3.Latitude FROM FCC_DATA_T t1, censusblocks_t t2, CityLongLat_t t3 WHERE substr(t1.BlockCode,0,12) = t2.blockcode AND t2.town = t3.town ORDER BY t2.town ASC"


#convert query results into dataframe
df_filtered_providers = pd.read_sql_query(qrystr, conn)
print(df_filtered_providers.head(10))

## Non Zero Providers

In [ ]:
qryStr2 = "SELECT DISTINCT t1.ProviderName,t1.MaxAdDown,t1.MaxAdUp, t2.town, t3.Longitude, t3.Latitude FROM FCC_DATA_T t1, censusblocks_t t2, CityLongLat_t t3 WHERE MaxAdDown != 0.0 AND MaxAdUp != 0.0 and substr(t1.BlockCode,0,12) = t2.blockcode AND t2.town = t3.town ORDER BY t2.town ASC"
df_filtered_non_zero_providers = pd.read_sql_query(qryStr2, conn)
print(df_filtered_non_zero_providers.head(10))


## Zero Providers

In [ ]:
qryStr3 = "SELECT DISTINCT t1.ProviderName,t1.MaxAdDown,t1.MaxAdUp, t2.town, t3.Longitude, t3.Latitude FROM FCC_DATA_T t1, censusblocks_t t2, CityLongLat_t t3 WHERE MaxAdDown = 0 AND MaxAdUp = 0 and substr(t1.BlockCode,0,12) = t2.blockcode AND t2.town = t3.town ORDER BY t2.town ASC"
df_filtered_zero_providers = pd.read_sql_query(qryStr3, conn)
print(df_filtered_zero_providers.head(10))

In [ ]:
def generateBaseMap(default_location=[42.360081, -71.058884]):
    base_map = folium.Map(location=default_location)
    return base_map

# download speed
base_map = generateBaseMap()

HeatMap(data=df_filtered_non_zero_providers [['Latitude', 'Longitude', 'MaxAdDown']].groupby(['Latitude', 'Longitude']).
        mean().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map) 

base_map.save('maxaddown.html')

# upload speed
base_map = generateBaseMap()

HeatMap(data=df_filtered_non_zero_providers [['Latitude', 'Longitude', 'MaxAdUp']].groupby(['Latitude', 'Longitude']).
        mean().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map) 

base_map.save('maxadup.html')

In [3]:
qryStr4 = "SELECT t2.Town as Town, t1.mhi as 'Median Income' FROM median_income_t t1, censusblocks_t t2 WHERE t2.blockcode = t1.ct10_id"
town_median_income  = pd.read_sql_query(qryStr4, conn)
print(town_median_income.head(10))

              Town  Median Income
0         ABINGTON         103527
1         ABINGTON          75089
2         ABINGTON          89978
3  ACUSHNET CENTER          84478
4  ACUSHNET CENTER          53705
5            ADAMS          40117
6            ADAMS          57426
7            ADAMS          53750
8      AGAWAM TOWN         101806
9      AGAWAM TOWN          72647
